In [1]:
# Load in one of the tables
df1 = spark.sql("select * from default.video_games_5")
df2 = spark.sql("select * from default.home_and_kitchen_5_small")
df3 = spark.sql("select * from default.books_5_small")

df = df1.union(df2).union(df3)

#df = df.sample(False, 0.005, seed=47)

df = df.cache()

print((df.count(), len(df.columns)))

(3487331, 12)

In [2]:
df.printSchema()

root
-- reviewID: long (nullable = true)
-- overall: double (nullable = true)
-- vote: integer (nullable = true)
-- verified: boolean (nullable = true)
-- reviewTime: string (nullable = true)
-- reviewerID: string (nullable = true)
-- asin: string (nullable = true)
-- reviewerName: string (nullable = true)
-- reviewText: string (nullable = true)
-- summary: string (nullable = true)
-- unixReviewTime: integer (nullable = true)
-- label: integer (nullable = true)

In [3]:
# For our intitial modeling efforts, we are not going to use the following features
drop_list = ['summary', 'asin', 'reviewID', 'reviewerID', 'summary', 'unixReviewTime','reviewTime', 'image', 'style', 'reviewerName']
df = df.select([column for column in df.columns if column not in drop_list])
df = df.na.drop(subset=["reviewText", "label"])
df.show(5)
print((df.count(), len(df.columns)))

+-------+----+--------+--------------------+-----+
overall|vote|verified| reviewText|label|
+-------+----+--------+--------------------+-----+
 5.0| 0| true|This game is a bi...| 0|
 4.0| 0| false|I played it a whi...| 0|
 3.0| 0| true| ok game.| 0|
 2.0| 0| true|found the game a ...| 0|
 5.0| 0| true|great game, I lov...| 0|
+-------+----+--------+--------------------+-----+
only showing top 5 rows

(3487331, 5)

In [4]:
# In Spark's MLLib, it's considered good practice to combine all the preprocessing steps into a pipeline.
# That way, you can run the same steps on both the training data, and testing data and beyond (new data)
# without copying and pasting any code.

# It is possible to run all of these steps one-by-one, outside of a Pipeline, if desired. But that's
# not how I am going to do it here.

from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer, VectorAssembler, IDF
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, NaiveBayes
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# We'll tokenize the text using a simple RegexTokenizer
tokenizer = RegexTokenizer(inputCol="reviewText", outputCol="words", pattern="\\W")

# Remove standard Stopwords
stopwordsRemover = StopWordsRemover(inputCol="words", outputCol="filtered")

# TODO: insert other clearning steps here (and put into the pipeline, of course!)
# E.g., n-grams? document length?


# Vectorize the sentences using simple BOW method. Other methods are possible:
# https://spark.apache.org/docs/2.2.0/ml-features.html#feature-extractors
tf = CountVectorizer(inputCol="filtered", outputCol="rawFeatures", vocabSize=2000, minTF=1, maxDF=0.40)

# Generate Inverse Document Frequency weighting
idf = IDF(inputCol="rawFeatures", outputCol="idfFeatures", minDocFreq=100)

# Combine all features into one final "features" column
assembler = VectorAssembler(inputCols=["verified", "overall", "idfFeatures"], outputCol="features")

# Machine Learning Algorithm
ml_alg  = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.0)

pipeline = Pipeline(stages=[tokenizer, stopwordsRemover, tf, idf, assembler, ml_alg])

paramGrid = ParamGridBuilder() \
    .addGrid(ml_alg.regParam, [0.3, 0.5, 0.7]) \
    .addGrid(ml_alg.elasticNetParam, [0.0]) \
    .addGrid(tf.minTF, [1, 100, 1000]) \
    .addGrid(tf.vocabSize, [500, 1000, 2500, 5000]) \
    .build()


In [5]:
# set seed for reproducibility
(trainingData, testData) = df.randomSplit([0.9, 0.1], seed = 47)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count:     " + str(testData.count()))

Training Dataset Count: 3138255
Test Dataset Count: 349076

In [6]:
pipelineFit = pipeline.fit(trainingData)

In [7]:
predictions = pipelineFit.transform(testData)
predictions.groupBy("prediction").count().show()

+----------+------+
prediction| count|
+----------+------+
 0.0|330157|
 1.0| 18919|
+----------+------+

In [8]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
pre_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
rec_evaluator = MulticlassClassificationEvaluator(metricName="weightedRecall")
pr_evaluator  = BinaryClassificationEvaluator(metricName="areaUnderPR")
auc_evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

#print("Test Accuracy       = %g" % (acc_evaluator.evaluate(predictions)))
#print("Test Precision      = %g" % (pre_evaluator.evaluate(predictions)))
#print("Test Recall         = %g" % (rec_evaluator.evaluate(predictions)))
#print("Test areaUnderPR    = %g" % (pr_evaluator.evaluate(predictions)))
print("Test areaUnderROC   = %g" % (auc_evaluator.evaluate(predictions)))

Test areaUnderROC = 0.843131

In [9]:
test_df = spark.sql("select * from default.reviews_kaggle")
kaggle_pred = pipelineFit.transform(test_df)
kaggle_pred.show(5)
kaggle_pred.groupBy("prediction").count().show()

+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
reviewID|overall|verified| reviewTime| reviewerID| asin| reviewerName| reviewText| summary|unixReviewTime| words| filtered| rawFeatures| idfFeatures| features| rawPrediction| probability|prediction|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
67000000| 3| true|05 16, 2012|A3IXM075VM1P9T|B007JYB3O2| nachtik|I would say these...| average reading| 1.3371264E9|[i, would, say, t...|[say, 5, regular,...|(2000,[1,7,13,30,...|(2000,[1,7,13,30,...|(2002,[0,1,3,9,15...|[1.54333457555824...|[0.82394895163181...| 0.0|
67000001| 5| true|12 25, 2012|A3LGZ7A3WSV3JJ| 985719745| MSP|WOW, DROPPIN DIME...|WOW.................| 1.3563936E9|[wow, droppin, di...|[wow, droppin, di...|(2000,[0,1,5,10,2...|(2000,[0,1,5,10,2...|(2002,[0,1,2,3,7,...|[1.46508050855351...|[0.81230849572719...| 0.0|
67000002| 5| true|09 18, 2012|A3BF5G7CJNIAG0|B002KXH7PQ| C. S. DeMore|You'll love the p...|Another Buggy Bar...| 1.3479264E9|[you, ll, love, t...|[ll, love, patter...|(2000,[0,1,3,9,10...|(2000,[0,1,3,9,10...|(2002,[0,1,2,3,5,...|[2.10275894194446...|[0.89117104429497...| 0.0|
67000003| 5| false|07 27, 2015|A2W41RTHSHYC4Q|B011LXVWRO| Light|For the majority ...|This is a complet...| 1.4379552E9|[for, the, majori...|[majority, person...|(2000,[1,9,15,20,...|(2000,[1,9,15,20,...|(2002,[1,3,11,17,...|[0.49895204428768...|[0.62221302613296...| 0.0|
67000004| 5| false|03 29, 2003| ACTBQZV1CJ9E8|9706061681|Richard Eastwood|MI MEJOR AMIGO..Q...|LE ACABA DE SUCED...| 1.048896E9|[mi, mejor, amigo...|[mi, mejor, amigo...|(2000,[1176,1570,...|(2000,[1176,1570,...|(2002,[1,1178,157...|[1.81134676345298...|[0.85952456404593...| 0.0|
+--------+-------+--------+-----------+--------------+----------+----------------+--------------------+--------------------+--------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

+----------+------+
prediction| count|
+----------+------+
 0.0|288024|
 1.0| 16960|
+----------+------+

In [10]:
# Download this and submit to Kaggle!
display(kaggle_pred.select(["reviewID", "prediction"]))

reviewID,prediction
67000000,0.0
67000001,0.0
67000002,0.0
67000003,0.0
67000004,0.0
67000005,0.0
67000006,0.0
67000007,0.0
67000008,0.0
67000009,0.0
